# How to Call the Ollama API (The 2-Way Demo)

This notebook mirrors the standalone scripts but keeps everything in one place, showing the full workflow:

- **0. Checking:** Is the server running & does our model exist?
- **1. Raw HTTP call:** The "manual" way using `requests`.
- **2. Python helper:** The "pro" way using the `ollama` library.

Run each cell in order. This notebook assumes `ollama serve` is running and you have already created the `kodzero-shorts` model from the previous video.

## 0. Checking: Is Ollama Running?

In [9]:
# We can use shell commands right from the notebook!
# Let's check if the server is responding.
!curl http://localhost:11434/

Ollama is running

### Check: Does our model 'kodzero-shorts' exist?

In [3]:
!pip install requests -q

In [11]:
# Let's list all local models to find our custom one.
# We'll use `requests` for this to get a clean JSON response.
import requests
import json

try:
    r = requests.get("http://localhost:11434/api/tags")
    r.raise_for_status() # Raise an error if the server returned one
    data = r.json()
    
    print("--- Available Local Models ---")
    models = [m['name'] for m in data['models']]
    print("\n".join(models))
    
    # Check for the model we created in the last episode
    if any("kodzero-shorts-json:latest" in m for m in models):
        print("\n✅ Success! Our 'kodzero-shorts' model is ready.")
    else:
        print("\n⚠️ Warning: 'kodzero-shorts' not found. Make sure you ran 'ollama create ...' first.")
        
except Exception as e:
    print(f"ERROR: Could not connect to Ollama. Is 'ollama serve' running?\n{e}")

--- Available Local Models ---
kodzero-shorts-json:latest
kodzero-shorts:latest
qwen2.5:7b-instruct-q4_K_M
gemma2:9b-instruct-q4_K_M
llama3.1:8b-instruct-q4_K_M
SpeakLeash/bielik-7b-instruct-v0.1-gguf:Q4_K_S
deepseek-r1:7b

✅ Success! Our 'kodzero-shorts' model is ready.


## 1. Method 1: The "Raw" Way (using `requests`)

This is the "manual" way. It's great for understanding what's happening under the hood. We will call the `/api/generate` endpoint.

In [12]:
# This is our payload. We use the exact model we created.
url = "http://localhost:11434/api/generate"

payload = {
    "model": "kodzero-shorts-json:latest",
    "prompt": "This is a transcript about installing Docker on a server...",
    "stream": False, # We want the full response at once
    "format": "json"
}

print("Sending request to Ollama...")
response = requests.post(url, json=payload, timeout=560)
response.raise_for_status() # Stop if we get an error

print(f"Status Code: {response.status_code}")
print("Request successful!")

Sending request to Ollama...
Status Code: 200
Request successful!


### Step 1.2: Analyze the Raw Response

This is the "Aha!" moment. The response from the server is a JSON object, but the part *we* want is *also* JSON, just nested inside as a string.

In [13]:
# Let's see what the server sent back.
raw_data = response.json()
raw_data

{'model': 'kodzero-shorts-json:latest',
 'created_at': '2025-11-14T19:10:34.311195528Z',
 'response': '{\n  "tytul": "Docker na serwerze - Instalacja w 5 minut! #shorts #viralshorts",\n  "opis": {\n    "zdanie_kluczowe": "Pokazuję prostą instalację Dockera na serwerze w kilka minut.",\n    "slowa_kluczowe": [\n      "docker",\n      "instalacja",\n      "serwer",\n      "linux",\n      "ubuntu",\n      "debian",\n      "centos",\n      "redhat"\n    ],\n    "hashtagi": [\n      "#shorts",\n      "#viralshorts",\n      "#docker",\n      "#serwer",\n      "#linux",\n      "#ubuntu"\n    ]\n  },\n  "tagi_seo": [\n    "instalacja docker na serwerze",\n    "docker na ubuntu",\n    "docker na debian",\n    "docker na centos",\n    "docker na redhat",\n    "serwer linux",\n    "linux serwer",\n    "ubuntu serwer",\n    "debian serwer",\n    "centos serwer",\n    "redhat serwer"\n  ],\n  "ocena_jakosci": "2.0/3",\n  "miniaturka_prompt": "scene: server room; hero: Docker logo on a terminal scre

### Step 1.3: Parse the *Nested* JSON

In [15]:
# Did you see that? The 'response' key isn't the final data.
# It's a STRING that *contains* our JSON.
# We have to parse it... again!

json_string = raw_data['response']
print(f"--- Raw JSON String from Model --- \n{json_string}")

# Now, parse the string to get a real Python object
final_data = json.loads(json_string)

print("\n--- ✅ Success! Parsed Title: ---")
# Use .get() to avoid errors if the key is missing or has different capitalization
print(final_data.get('tytul') or final_data.get('Tytul'))

--- Raw JSON String from Model --- 
{
  "tytul": "Docker na serwerze - Instalacja w 5 minut! #shorts #viralshorts",
  "opis": {
    "zdanie_kluczowe": "Pokazuję prostą instalację Dockera na serwerze w kilka minut.",
    "slowa_kluczowe": [
      "docker",
      "instalacja",
      "serwer",
      "linux",
      "ubuntu",
      "debian",
      "centos",
      "redhat"
    ],
    "hashtagi": [
      "#shorts",
      "#viralshorts",
      "#docker",
      "#serwer",
      "#linux",
      "#ubuntu"
    ]
  },
  "tagi_seo": [
    "instalacja docker na serwerze",
    "docker na ubuntu",
    "docker na debian",
    "docker na centos",
    "docker na redhat",
    "serwer linux",
    "linux serwer",
    "ubuntu serwer",
    "debian serwer",
    "centos serwer",
    "redhat serwer"
  ],
  "ocena_jakosci": "2.0/3",
  "miniaturka_prompt": "scene: server room; hero: Docker logo on a terminal screen; anchors: Linux distro logos (Ubuntu, Debian, CentOS, Redhat); overlay (PL): 'Docker na serwerze'; st

## 2. Method 2: The "Pro" Way (using `ollama`)

That was a lot of work. The official `ollama` library makes this much cleaner.

In [16]:
!pip install ollama -q

In [17]:
import ollama

print("Sending request with the 'ollama' library...")

# One single command!
# We use .generate() because our Modelfile is simple
response = ollama.generate(
    model='kodzero-shorts-json:latest', 
    prompt='This is a transcript about installing Docker on a server...'
)

print("Request successful!")

Sending request with the 'ollama' library...
Request successful!


### Step 2.2: Analyze and Parse (Still Needed!)

The library simplifies the *call*, but the *response* from `/api/generate` is the same. Our model is still designed to output a JSON string, so the final parsing step is identical.

In [18]:
# The library gives us a clean dictionary
response

GenerateResponse(model='kodzero-shorts-json:latest', created_at='2025-11-14T19:12:32.716308785Z', done=True, done_reason='stop', total_duration=57492418126, load_duration=139120547, prompt_eval_count=977, prompt_eval_duration=194100098, eval_count=318, eval_duration=56637641945, response='{\n  "tytul": "Docker na serwerze - instalacja od podstaw #shorts #viralshorts",\n  "opis": {\n    "zdanie_kluczowe": "Pokazuję, jak zainstalować Docker na serwerze i rozpocząć pracę z kontenerami.",\n    "slowa_kluczowe": [\n      "docker",\n      "serwer",\n      "kontenery",\n      "instalacja",\n      "ubuntu",\n      "debian",\n      "centos",\n      "redhat"\n    ],\n    "hashtagi": [\n      "#shorts",\n      "#viralshorts",\n      "#docker",\n      "#serwer",\n      "#kontenery",\n      "#linux"\n    ]\n  },\n  "tagi_seo": [\n    "docker na serwerze",\n    "instalacja docker",\n    "kontenery na serwerze",\n    "ubuntu docker",\n    "debian docker",\n    "centos docker",\n    "redhat docker",\n

In [19]:
# Look familiar? We still get a JSON string in the 'response' key.
json_string = response['response']
print(f"--- Raw JSON String from Model --- \n{json_string}")

final_data = json.loads(json_string)

print("\n--- ✅ Success! Parsed Title: ---")
print(final_data.get('tytul') or final_data.get('Tytul'))

--- Raw JSON String from Model --- 
{
  "tytul": "Docker na serwerze - instalacja od podstaw #shorts #viralshorts",
  "opis": {
    "zdanie_kluczowe": "Pokazuję, jak zainstalować Docker na serwerze i rozpocząć pracę z kontenerami.",
    "slowa_kluczowe": [
      "docker",
      "serwer",
      "kontenery",
      "instalacja",
      "ubuntu",
      "debian",
      "centos",
      "redhat"
    ],
    "hashtagi": [
      "#shorts",
      "#viralshorts",
      "#docker",
      "#serwer",
      "#kontenery",
      "#linux"
    ]
  },
  "tagi_seo": [
    "docker na serwerze",
    "instalacja docker",
    "kontenery na serwerze",
    "ubuntu docker",
    "debian docker",
    "centos docker",
    "redhat docker",
    "serwer linux",
    "kontenery linux",
    "docker instalacja od podstaw"
  ],
  "ocena_jakosci": "2.0/3",
  "miniaturka_prompt": "scene: server room; hero: Docker logo on a terminal screen; anchors: Ubuntu/Debian/CentOS/Redhat logos; overlay (PL): 'Docker na serwerze'; styling: h

## Bonus: The Chat Endpoint

My production script uses the `/api/chat` endpoint. The `ollama` library makes this easy too. The main difference is the response structure (`response['message']['content']`).

In [ ]:
prompt_text = "This is a transcript about installing Docker on a server..."

# This is how you'd call the /api/chat endpoint
response_chat = ollama.chat(
    model='kodzero-shorts-json',
    messages=[
        {'role': 'user', 'content': prompt_text}
    ]
)

# The structure is slightly different:
json_string_chat = response_chat['message']['content']
print(f"--- Raw JSON String from Chat Model --- \n{json_string_chat}")

final_data_chat = json.loads(json_string_chat)

print("\n--- ✅ Success! Parsed Title from Chat: ---")
print(final_data_chat.get('tytul') or final_data_chat.get('Tytul'))

--- Raw JSON String from Chat Model --- 
[Tytul]
Instaluj Docker na serwerze #shorts #viralshorts

[Opis]
Możesz łatwo uruchomić aplikacje w chmurze za pomocą Dockera.

 
docker,
server,
chmura,
aplikacja,
kontener,
komputer,
system,
linux,
programowanie,
cyfrowy,
zarządzanie,
serwerowe,

#shorts #viralshorts #docker #server #cloudcomputing #devops #programming #linux

[Tagi]
ollama local setup, open webui panel, docker compose start, wybor modelu llm, konfiguracja parametrów, lokalny chat ai, docker compose deploy, bezpieczenstwo danych, comfyui cpu workflow, generowanie miniatur, tailscale vpn basic, open source modele, zarzadzanie konteks tem, publikacja shorts

[Kiedy]
2.1/3

[Miniaturka]
scene: server room with a rack of servers; hero: "Docker Installed"; anchors: a Docker logo on the server screen, a container running in the background; overlay: "Użyj Dockera"; callouts: "Zainstaluj teraz"; styling: vertical portrait, high contrast, minimal clutter.


JSONDecodeError: Expecting value: line 1 column 2 (char 1)